In [1]:
from sentence_transformers import SentenceTransformer, util
import pm4py
from pm4py.objects.bpmn.obj import BPMN
import torch
import torch.nn.functional as F
import numpy as np
import pandas as pd

In [2]:
bpmn = pm4py.read_bpmn('X_G01_text.bpmn')
temp=[]
for flow in bpmn.get_flows():
    temp.append([flow.source,flow.target,flow.get_name()])

for rel in temp:
    if(isinstance(rel[1],BPMN.Gateway) and len(rel[1].get_in_arcs())>1):
        rel[1]=rel[1].get_out_arcs()[0].target
        while(isinstance(rel[1],BPMN.Gateway)):
            rel[1]=rel[1].get_out_arcs()[0].target

temp1=[]
for rel in temp:
    if(rel[0]==rel[1]):
        continue
    elif(isinstance(rel[0],BPMN.Gateway) and len(rel[0].get_in_arcs())>1):
        continue
    else:
        temp1.append(rel)
df_BPMN = pd.DataFrame(columns=['Source', 'Target', 'Relation'])
for rel in temp1:
    if(isinstance(rel[1],BPMN.EndEvent) or isinstance(rel[0],BPMN.StartEvent)):
        continue
    if(isinstance(rel[0],BPMN.ExclusiveGateway)):
        # temp2.append([rel[0].get_in_arcs()[0].source, rel[1]])
        df_BPMN = df_BPMN.append({'Source': rel[0].get_in_arcs()[0].source.name, 'Target': rel[1].name, 'Relation': 'XOR_gateway'}, ignore_index=True)
        df_BPMN = df_BPMN.append({'Source': rel[2], 'Target': rel[1].name, 'Relation': 'XOR_condition'}, ignore_index=True)
        # print([rel[0].get_in_arcs()[0].source, rel[1]])
    elif(isinstance(rel[1],BPMN.ExclusiveGateway)):
        continue
    elif(isinstance(rel[0],BPMN.ParallelGateway)):
        df_BPMN = df_BPMN.append({'Source': rel[0].get_in_arcs()[0].source.name, 'Target': rel[1].name, 'Relation': 'parallel_gateway'}, ignore_index=True)
    elif(isinstance(rel[1],BPMN.ParallelGateway)):
        continue
    else:
        df_BPMN = df_BPMN.append({'Source': rel[0].name, 'Target': rel[1].name, 'Relation': 'Flow'}, ignore_index=True)

In [3]:
# log = pm4py.read_xes("R_j04_event.xes")
# bpmn_event = pm4py.discover_bpmn_inductive(log, activity_key='concept:name', case_id_key='case:concept:name', timestamp_key='time:timestamp')
bpmn_event = pm4py.read_bpmn('X_G01_event.bpmn')
temp=[]
for flow in bpmn_event.get_flows():
    temp.append([flow.source,flow.target,flow.get_name()])
for rel in temp:
    if(isinstance(rel[1],BPMN.Gateway) and len(rel[1].get_in_arcs())>1):
        rel[1]=rel[1].get_out_arcs()[0].target
        while(isinstance(rel[1],BPMN.Gateway)):
            rel[1]=rel[1].get_out_arcs()[0].target

temp1=[]
for rel in temp:
    if(rel[0]==rel[1]):
        continue
    elif(isinstance(rel[0],BPMN.Gateway) and len(rel[0].get_in_arcs())>1):
        continue
    else:
        temp1.append(rel)
df_BPMN_event = pd.DataFrame(columns=['Source', 'Target', 'Relation'])
for rel in temp1:
    if(isinstance(rel[1],BPMN.EndEvent) or isinstance(rel[0],BPMN.StartEvent)):
        continue
    if(isinstance(rel[0],BPMN.ExclusiveGateway)):
        # temp2.append([rel[0].get_in_arcs()[0].source, rel[1]])
        df_BPMN_event = df_BPMN_event.append({'Source': rel[0].get_in_arcs()[0].source.name, 'Target': rel[1].name, 'Relation': 'XOR_gateway'}, ignore_index=True)
        df_BPMN_event = df_BPMN_event.append({'Source': rel[2], 'Target': rel[1].name, 'Relation': 'XOR_condition'}, ignore_index=True)
        # print([rel[0].get_in_arcs()[0].source, rel[1]])
    elif(isinstance(rel[1],BPMN.ExclusiveGateway)):
        continue
    elif(isinstance(rel[0],BPMN.ParallelGateway)):
        df_BPMN_event = df_BPMN_event.append({'Source': rel[0].get_in_arcs()[0].source.name, 'Target': rel[1].name, 'Relation': 'parallel_gateway'}, ignore_index=True)
    elif(isinstance(rel[1],BPMN.ParallelGateway)):
        continue
    else:
        df_BPMN_event = df_BPMN_event.append({'Source': rel[0].name, 'Target': rel[1].name, 'Relation': 'Flow'}, ignore_index=True)

In [4]:
list_event=[]
for row in df_BPMN_event.itertuples(index=False):
    if(row.Relation!='XOR_condition'):
        list_event.append((row.Source,row.Target,row.Relation))
list_BPMN=[]
for row in df_BPMN.itertuples(index=False):
    if(row.Relation!='XOR_condition'):
        list_BPMN.append((row.Source,row.Target,row.Relation))
dif_gold=set(list(set(list_BPMN).difference(set(list_event)))+list(set(list_event).difference(set(list_BPMN))))

In [5]:
log = pm4py.read_xes("X_G01_event.xes")
bpmn_event = pm4py.discover_bpmn_inductive(log, activity_key='concept:name', case_id_key='case:concept:name', timestamp_key='time:timestamp')
# bpmn_event = pm4py.read_bpmn('R_j04_event.bpmn')
temp=[]
for flow in bpmn_event.get_flows():
    temp.append([flow.source,flow.target,flow.get_name()])
for rel in temp:
    if(isinstance(rel[1],BPMN.Gateway) and len(rel[1].get_in_arcs())>1):
        rel[1]=rel[1].get_out_arcs()[0].target
        while(isinstance(rel[1],BPMN.Gateway)):
            rel[1]=rel[1].get_out_arcs()[0].target

temp1=[]
for rel in temp:
    if(rel[0]==rel[1]):
        continue
    elif(isinstance(rel[0],BPMN.Gateway) and len(rel[0].get_in_arcs())>1):
        continue
    else:
        temp1.append(rel)
df_BPMN_event = pd.DataFrame(columns=['Source', 'Target', 'Relation'])
for rel in temp1:
    if(isinstance(rel[1],BPMN.EndEvent) or isinstance(rel[0],BPMN.StartEvent)):
        continue
    if(isinstance(rel[0],BPMN.ExclusiveGateway)):
        # temp2.append([rel[0].get_in_arcs()[0].source, rel[1]])
        df_BPMN_event = df_BPMN_event.append({'Source': rel[0].get_in_arcs()[0].source.name, 'Target': rel[1].name, 'Relation': 'XOR_gateway'}, ignore_index=True)
        df_BPMN_event = df_BPMN_event.append({'Source': rel[2], 'Target': rel[1].name, 'Relation': 'XOR_condition'}, ignore_index=True)
        # print([rel[0].get_in_arcs()[0].source, rel[1]])
    elif(isinstance(rel[1],BPMN.ExclusiveGateway)):
        continue
    elif(isinstance(rel[0],BPMN.ParallelGateway)):
        df_BPMN_event = df_BPMN_event.append({'Source': rel[0].get_in_arcs()[0].source.name, 'Target': rel[1].name, 'Relation': 'parallel_gateway'}, ignore_index=True)
    elif(isinstance(rel[1],BPMN.ParallelGateway)):
        continue
    else:
        df_BPMN_event = df_BPMN_event.append({'Source': rel[0].name, 'Target': rel[1].name, 'Relation': 'Flow'}, ignore_index=True)

c:\Users\20235451\.conda\envs\trochtest\lib\site-packages\pm4py\util\dt_parsing\parser.py:78: UserWarning: ISO8601 strings are not fully supported with strpfromiso for Python versions below 3.11
  "ISO8601 strings are not fully supported with strpfromiso for Python versions below 3.11")


parsing log, completed traces ::   0%|          | 0/1000 [00:00<?, ?it/s]

In [6]:
model = SentenceTransformer("Maite89/Roberta_finetuning_semantic_similarity_stsb_multi_mt")
Activities_text=[]
relations=[('select a course', 'tell you the courses and dates', 'XOR_gateway'),
('slots are free', 'tell you the courses and dates', 'XOR_condition'),
('tell you the courses and dates', 'check if you already have an account at the sports institute', 'Flow'),
('check if you already have an account at the sports institute', 'check whether you are registered at an eligible university', 'XOR_gateway'),
('do not have an account at the sports institute', 'check whether you are registered at an eligible university', 'XOR_condition'),
('check whether you are registered at an eligible university', 'register your account', 'XOR_gateway'),
('are registered at an eligible university', 'register your account', 'XOR_condition'),
('check whether you are registered at an eligible university', 'request an activation', 'XOR_gateway'),
('are not registered at an eligible university', 'request an activation', 'XOR_condition'),
('request an activation', 'wait for a response', 'Flow'),
('have an account', 'log into it', 'Flow'),
('log into it', 'tweet to friends who might want to join you', 'parallel_gateway'),
('log into it', 'complete the registration for the course', 'parallel_gateway'),
('complete the registration for the course', 'provide the payment information', 'Flow'),
('tweet to friends who might want to join you', 'receive a course ticket', 'Flow'),
('provide the payment information', 'receive a course ticket', 'Flow')]
df_text = pd.DataFrame(columns=['Source', 'Target', 'Relation'])
for relation in relations:
    df_text = df_text.append({'Source': relation[0], 'Target': relation[1], 'Relation': relation[2]}, ignore_index=True)
    Activities_text.append(relation[1])
    if relation[2]!='XOR_condition':
        Activities_text.append(relation[0])
queries=list(set(Activities_text))

In [7]:
Activities_BPMN=[]
for node in bpmn_event.get_nodes():
    if(isinstance(node,BPMN.EndEvent) or isinstance(node,BPMN.StartEvent)):
        continue
    if(isinstance(node,BPMN.Activity) or isinstance(node,BPMN.Event)):
        Activities_BPMN.append(node.name)

# entity match
df = pd.DataFrame(columns=['text', 'BPMN', 'score'])
temp_list=[]
for query in queries:
    query_embedding = torch.FloatTensor(model.encode(query))
    scores=[]
    for activity in Activities_BPMN:
        activity_embedding = torch.FloatTensor(model.encode(activity))
        cos_sim = F.cosine_similarity(query_embedding, activity_embedding, dim=0)
        scores.append(cos_sim.item())
    # print(scores)
    max_score_pos=np.argmax(scores)
    max_score=np.max(scores)
    if(max_score<0.5):
        df = df.append({'text': query, 'BPMN': "No match found", 'score': max_score}, ignore_index=True)
    else:
        if(max_score_pos in temp_list):
            score_tep=df.loc[df['BPMN']==Activities_BPMN[max_score_pos]]['score'].tolist()[0]
            if(max_score>score_tep):
                df.loc[df['BPMN']==Activities_BPMN[max_score_pos],'BPMN']="No match found"
                df = df.append({'text': query, 'BPMN': Activities_BPMN[max_score_pos], 'score': max_score}, ignore_index=True)
            else:
                df = df.append({'text': query, 'BPMN': "No match found", 'score': max_score}, ignore_index=True)
        else:
            temp_list.append(max_score_pos)
            df = df.append({'text': query, 'BPMN': Activities_BPMN[max_score_pos], 'score': max_score}, ignore_index=True)
for i in range(len(Activities_BPMN)):
    if i not in temp_list:
        df = df.append({'text': "No match found", 'BPMN': Activities_BPMN[i], 'score': 0}, ignore_index=True)

In [8]:
# replace
df_text_new = pd.DataFrame(columns=['Source', 'Target', 'Relation'])
for row in df_text.itertuples(index=False):
    source=row.Source
    target=row.Target
    relation=row.Relation
    if(source!="No match found"):
        if(len(df.loc[df['text']==source]['BPMN'].tolist())>0):
            if(df.loc[df['text']==source]['BPMN'].tolist()[0]!="No match found"):
                source=df.loc[df['text']==source]['BPMN'].tolist()[0]
    if(target!="No match found"):
        if(len(df.loc[df['text']==target]['BPMN'].tolist())>0):
            if(df.loc[df['text']==target]['BPMN'].tolist()[0]!="No match found"):
                target=df.loc[df['text']==target]['BPMN'].tolist()[0]
    df_text_new = df_text_new.append({'Source': source, 'Target': target, 'Relation': relation}, ignore_index=True)

list_text=[]
for row in df_text_new.itertuples(index=False):
    if(row.Relation!='XOR_condition'):
        list_text.append((row.Source,row.Target,row.Relation))
list_event=[]
for row in df_BPMN_event.itertuples(index=False):
    if(row.Relation!='XOR_condition'):
        list_event.append((row.Source,row.Target,row.Relation))
dif=set(list(set(list_text).difference(set(list_event)))+list(set(list_event).difference(set(list_text))))
precision=len(dif.intersection(dif_gold))/(len(dif))
recall=len(dif.intersection(dif_gold))/len(dif_gold)
print('precision:',precision)
print('recall:',recall)

precision: 0.2777777777777778
recall: 0.5555555555555556


In [9]:
# detected deviation relationships
dif

{('Check if account  at sports institute available',
  'Log into account',
  'XOR_gateway'),
 ('Check if registered at an eligible university',
  'Register with account',
  'XOR_gateway'),
 ('Check if registered at an eligible university',
  'Register with account',
  'parallel_gateway'),
 ('Check if registered at an eligible university',
  'Send request for activation',
  'XOR_gateway'),
 ('Check if registered at an eligible university',
  'Send request for activation',
  'parallel_gateway'),
 ('Log into account',
  'complete the registration for the course',
  'parallel_gateway'),
 ('Log into account', 'connect to twitter account', 'Flow'),
 ('Log into account', 'invite friends to course', 'parallel_gateway'),
 ('Register with account', 'Log into account', 'Flow'),
 ('Select course', 'Select course', 'XOR_gateway'),
 ('Select one date',
  'Check if account  at sports institute available',
  'Flow'),
 ('Show courses and dates',
  'Check if account  at sports institute available',
  'F